<a href="https://colab.research.google.com/github/Mohan0332/Capstone/blob/main/CNN(exis)%20with%20augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementation of the exisiting model

In [1]:
import numpy as np
from PIL import Image
import os

In [2]:
# function to convert a image to grayscale 
def rgb_to_gray(rgb_img):
    # Convert the image to a NumPy array
    rgb_array = np.array(rgb_img)

    # Create an empty grayscale image array with the same dimensions as the RGB image
    gray_array = np.empty(rgb_array.shape[:2], dtype=np.uint8)

    # Calculate the luminance value for each pixel in the RGB image and assign it to the grayscale image array
    for i in range(rgb_array.shape[0]):
        for j in range(rgb_array.shape[1]):
            r, g, b = rgb_array[i,j]
            luminance = int(0.2989 * r + 0.5870 * g + 0.1140 * b)
            gray_array[i,j] = luminance
    
    gray_img = Image.fromarray(gray_array)
    return gray_img

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = []
labels = []
classes = 2
cur_path = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB'

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((100,100))
            img = rgb_to_gray(image)
            img = np.array(img)
            img = img.reshape((100, 100, 1))
            #image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(img)
            labels.append(i)
        except:
            print("Error loading image")

In [5]:
data = np.array(data)
labels = np.array(labels)

In [6]:
print(data.shape, labels.shape)

(490, 100, 100, 1) (490,)


In [46]:
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [47]:
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.22, random_state=42)

In [48]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(382, 100, 100, 1) (108, 100, 100, 1) (382,) (108,)


In [49]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [59]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(100,100,1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [60]:
#Compilation of the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 98, 98, 16)        160       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 10, 10, 64)      

In [80]:
from keras.preprocessing.image import ImageDataGenerator
EPOCHS = 20
BS = 32
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=45,
	width_shift_range=0.2, height_shift_range=0.2,
	horizontal_flip=True)
# train the network
H = model.fit_generator(aug.flow(X_train, y_train, batch_size=BS),
	validation_data=(X_test,y_test), steps_per_epoch=len(X_train) // BS,
	epochs=EPOCHS)

Epoch 1/20


<ipython-input-80-91e032fe9352>:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(aug.flow(X_train, y_train, batch_size=BS),


11/11 [==============================] - 6s 571ms/step - loss: 0.4617 - accuracy: 0.8000 - val_loss: 0.5429 - val_accuracy: 0.7593
Epoch 2/20
11/11 [==============================] - 3s 292ms/step - loss: 0.4456 - accuracy: 0.8086 - val_loss: 0.5427 - val_accuracy: 0.7222
Epoch 3/20
11/11 [==============================] - 5s 483ms/step - loss: 0.4858 - accuracy: 0.7686 - val_loss: 0.5183 - val_accuracy: 0.7037
Epoch 4/20
11/11 [==============================] - 3s 283ms/step - loss: 0.4577 - accuracy: 0.7841 - val_loss: 0.5190 - val_accuracy: 0.7593
Epoch 5/20
11/11 [==============================] - 5s 462ms/step - loss: 0.4333 - accuracy: 0.8143 - val_loss: 0.5308 - val_accuracy: 0.7315
Epoch 6/20
11/11 [==============================] - 3s 281ms/step - loss: 0.4453 - accuracy: 0.8000 - val_loss: 0.6014 - val_accuracy: 0.7315
Epoch 7/20
11/11 [==============================] - 3s 286ms/step - loss: 0.4560 - accuracy: 0.7971 - val_loss: 0.6274 - val_accuracy: 0.7593
Epoch 8/20
11/11 

In [81]:
test_data = []
test_labels = []
classes = 2
cur_path = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB'

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'test',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((100,100))
            img = rgb_to_gray(image)
            img = np.array(img)
            img = img.reshape((100, 100, 1))
            #sim = Image.fromarray(image)
            test_data.append(img)
            test_labels.append(i)
        except:
            print("Error loading image")
     


In [82]:
x_test=np.array(test_data)

In [83]:
x_test.shape

(57, 100, 100, 1)

In [84]:
y_pred_prob = model.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)

2/2 [==============================] - 0s 55ms/step


In [85]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_labels,y_pred ))

0.7368421052631579
